In [ ]:
from youtube_transcript_api import YouTubeTranscriptApi
from pytube import extract
from pytube import YouTube

In [1]:
api_key = "vo1u96ecfrozurau"

import logging
from kiteconnect import KiteConnect

logging.basicConfig(level=logging.DEBUG)

kite = KiteConnect(api_key=api_key)

print(kite.login_url())

https://kite.zerodha.com/connect/login?api_key=vo1u96ecfrozurau&v=3


In [2]:
instruments = kite.instruments()

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments HTTP/1.1" 200 1277066


In [3]:
stocks = []
for i in instruments:
    if i["exchange"] == "NSE" and i["segment"] == "NSE" and i["instrument_type"] == "EQ":
        stocks.append(i)

In [4]:
len(stocks)

5545

In [6]:
dict_pairs = {}
for i in stocks:
    dict_pairs[i["name"]] = i["tradingsymbol"]

In [29]:
from langchain.llms import OpenAI
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from dotenv import load_dotenv
load_dotenv()

llm = OpenAI(temperature=0.5)
tools = load_tools(["serpapi"], llm=llm)
agent = initialize_agent(tools, llm, verbose=True, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION)

In [36]:
response = agent.run("what is the ticker or trading symbol for Gujarat Themis Biosyn Ltd")



> Entering new AgentExecutor chain...
 I should look for the company's financial information
Action: Search
Action Input: "Gujarat Themis Biosyn Ltd stock symbol"
Observation: Guj Themis, 801.10, -0.11 ; Sun Pharma, 1,134.90, -0.86 ; Cipla, 1,236.40, -0.37 ; Dr Reddys Labs, 5,874.45, -0.11 ...
Thought: I should look for the ticker symbol for Gujarat Themis Biosyn Ltd
Action: Search
Action Input: "Gujarat Themis Biosyn Ltd ticker symbol"
Observation: Find the latest Gujarat Themis Biosyn Limited (GUJTHEM.BO) stock quote, history, news and other vital information to help you with your stock trading and ...
Thought: The ticker symbol for Gujarat Themis Biosyn Ltd is GUJTHEM.BO
Final Answer: The ticker symbol for Gujarat Themis Biosyn Ltd is GUJTHEM.BO

> Finished chain.


In [32]:
print(response)

The stock ticker for ICICI Bank is ICICIBANK.


In [ ]:
video_url = "https://www.youtube.com/watch?v=5hhyZtBRwNQ"

youtube = YouTube(video_url)

author = youtube.author
video_id=extract.video_id(video_url)
print(author)
print(video_id)

In [ ]:
# using the srt variable with the list of dictionaries
# obtained by the .get_transcript() function
srt = YouTubeTranscriptApi.get_transcript(video_id)

transcript = ""
for i in srt:
    transcript = transcript + " " + i["text"]

In [ ]:
print(transcript)

In [ ]:
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv
load_dotenv()

chat = ChatOpenAI(temperature=0, model_name='gpt-4', request_timeout=120)

In [ ]:
from langchain.schema import (
    HumanMessage,
    SystemMessage
)

messages = [
    SystemMessage(content="You are a Financial Analyst. Your task is to identify the company stock and the context of claims made on the stock by a Financial Influencer. \
                  Please separate out the theoretical and quantitative analysis from the response. Report the response in JSON format with keys company_name, claims, theoretical_analysis and quantitative_analysis."),
    HumanMessage(content=transcript)
]

In [ ]:
response = chat(messages)

In [ ]:
eval(response.content)

In [ ]:
import json

with open('data.json', 'w', encoding='utf-8') as f:
    json.dump(response.content, f, ensure_ascii=False, indent=4)

In [ ]:
with open('data.json', 'r', encoding='utf-8') as f:
    result = eval(json.load(f))

In [ ]:
result.keys()

In [ ]:
result["company_name"]

In [ ]:
result["claims"]

In [ ]:
result["theoretical_analysis"]

In [ ]:
result["quantitative_analysis"]